In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from src.load_scripts import load_log, load_item

In [ ]:
data_path = Path('data/')

retrain = False

if retrain:
    item = load_item(data_path)
    log = load_log(data_path)
    #item.to_csv(data_path / 'cached_item.csv')
    #log.to_csv(data_path / 'cached_log.csv')
else:
    # load from cache
    item = pd.read_csv(data_path / 'cached_item.csv', index_col=0)

    # load from cache
    log = pd.read_csv(data_path / 'cached_log.csv')
    log['time'] = pd.to_datetime(log['time'])
    log.set_index('time', inplace=True)

In [ ]:
new_log = []
counts = log['user'].value_counts()

for file_path in (data_path / 'edulint').glob("*.json"):
    start, end = file_path.stem.split('_')[2].split('-')
    start, end = int(start), int(end)
    selected_counts = counts[(counts >= start) & (counts < end)]
    log_slice = pd.DataFrame(log.query('user in @selected_counts.index'))
    log_slice['linter_messages'] = [' '.join(alist) for alist in json.load(open(file_path, 'r'))]
    new_log.append(log_slice)

new_log = pd.concat(new_log)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.005)
vectors = vectorizer.fit_transform(new_log['linter_messages'])

new_log['linter_messages'] = list(map(np.array, vectors.toarray().tolist()))

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
with open(Path('data/edulint/results.txt'), 'r') as f:
    results = f.read().lower()
    
feature_descriptions = []
for feature_name in vectorizer.get_feature_names_out():
    begin = results.find(feature_name)
    feature_descriptions.append(results[begin:results.find('_"', begin)])

json.dump(feature_descriptions, open(Path('data/edulint/features.json'), 'w'))

In [ ]:
feature_descriptions[13]

In [ ]:
new_log

In [ ]:
new_log['linter_messages'] = new_log['linter_messages'].apply(lambda x: x.tolist())
new_log.to_csv(data_path / 'dataset.csv')